# Dynamic inter-prism calibration results

Notebook which generates the results of the dynamic inter-prism calibration

In [1]:
import numpy as np
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.resection_functions as theodo_r
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.transform import Rotation as R

In [8]:
# This cell gathers result for the inter-prism distance resection
theodo_r = importlib.reload(theodo_r)
theodo_u = importlib.reload(theodo_u)

path = "../data/20221129-1/"
name_param_file = "f-3-2-2-4-4-0-L_"
#name_param_file = "ns_sync_p_per_"
robot_type = "Warthog"    # Warthog or HD2 or simulation
debug = True
save_results = True

if robot_type!="simulation":
    i = path+"filtered_prediction/"+name_param_file
    j = path+"total_stations/GCP.txt"
else:
    i = path+"total_stations/"+name_param_file
    j = ""
file_sensors = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/calibration_results.csv",'')
extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
k = extrinsic_calibration_results
if robot_type=="Warthog":
    l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
else:
    if robot_type=="HD2":
        l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2]
    else:
        if robot_type=="simulation":
            l = [0.01,1,2,4,7,8,9,10,11,12]
m = ''

Tf_results = []
Error_results = []
Error_results_cp = []
RF = ['','']
print("File: ",i)
for speed in l:
    print("Speed: ", speed)
    dist_error,_,errors_prism,_, errors_exp, TF_list = theodo_r.one_inter_prism_resection_advanced(Inter_distance=k,
                            file_name=i, file_name_marker=j, RF=RF, robot=robot_type, rate=10, prior = "A", velocity_outlier = speed,
                            threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)
    if len(dist_error)>1:
        RF1 = TF_list[0][1]
        RF2 = TF_list[0][2]
        r1 = R.from_matrix(RF1[0:3,0:3])
        r2 = R.from_matrix(RF2[0:3,0:3])
        euler1 = r1.as_euler('xyz', degrees=True)
        euler2 = r2.as_euler('xyz', degrees=True)
        if debug:
            print(RF1[0:3,3],RF2[0:3,3])
            print(euler1,euler2)
            print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
            print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
        RF = [RF1,RF2]
        Tf_results.append([TF_list[0][1],TF_list[0][2]])
        Error_results.append(np.median(errors_exp))
        Error_results_cp.append(np.median(errors_prism))
    else:
        Tf_results.append(['',''])
        Error_results.append(100000)
        Error_results_cp.append(100000)

arr_error = np.array(Error_results)
index_error = theodo_u.check_if_converge_well(arr_error, Tf_results, debug)

if index_error!=-1:

    errors_linear_dynamic_local = []
    errors_linear_dynamic_cp_local = []
    errors_linear_dynamic_cp_exp_local = []
    Tf_results_bis = []

    Error_results_bis = []
    Error_results_cp_bis = []
    RF = Tf_results[index_error]
    print("File: ",i)
    for speed in l:
        print("Speed: ", speed)
        dist_error,_,errors_prism,_, errors_exp, TF_list = theodo_r.one_inter_prism_resection_advanced(Inter_distance=k,
                                file_name=i, file_name_marker=j, RF=RF, robot=robot_type, rate=10, prior = "A", velocity_outlier = speed,
                                threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)
        errors_linear_dynamic_local.append(dist_error)
        errors_linear_dynamic_cp_local.append(errors_prism)
        errors_linear_dynamic_cp_exp_local.append(errors_exp)

        if len(dist_error)>1:
            RF1 = TF_list[0][1]
            RF2 = TF_list[0][2]
            r1 = R.from_matrix(RF1[0:3,0:3])
            r2 = R.from_matrix(RF2[0:3,0:3])
            euler1 = r1.as_euler('xyz', degrees=True)
            euler2 = r2.as_euler('xyz', degrees=True)
            if debug:
                print(RF1[0:3,3],RF2[0:3,3])
                print(euler1,euler2)
                print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
                print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
            Tf_results_bis.append([TF_list[0][1],TF_list[0][2]])
            Error_results_bis.append(np.median(errors_exp))
            Error_results_cp_bis.append(np.median(errors_prism))
        else:
            Tf_results_bis.append(['',''])
            Error_results_bis.append(100000)
            Error_results_cp_bis.append(100000)

    arr_error_bis = np.array(Error_results_bis)
    index_error_bis = theodo_u.check_if_converge_well(arr_error_bis, Tf_results_bis, debug)
    print("CP error minimum: ", Error_results_cp_bis[index_error_bis])

    Tf_list_minim = []
    T1 = np.identity(4)
    Tf_list_minim.append(T1.tolist())
    for i in Tf_results_bis[index_error_bis]:
        Tf_list_minim.append(i.tolist())

    if save_results and index_error_bis!=-1:
        theodo_u.save_error_list_to_file(errors_linear_dynamic_cp_local[index_error_bis], path+"list_error/errors_linear_dynamic_cp_4dof.csv")
        theodo_u.save_error_list_to_file(errors_linear_dynamic_cp_exp_local[index_error_bis], path+"list_error/errors_linear_dynamic_cp_experiment_4dof.csv")
        theodo_u.save_tf_list_to_file_multi([Tf_list_minim], path+"list_tf/TF_list_linear_dynamic_4dof.csv")


File:  ../data/20221129-1/filtered_prediction/f-3-2-2-4-4-0-L_
Speed:  0.01
Selected points:  3
Results done !
Speed:  0.05
Selected points:  7
Results done !
Speed:  0.1
Selected points:  9
4-DOF
Time [s]:  0.10159850120544434
Jump detected !
Time [s]:  0.05721712112426758
Results done !
[ 3.34809891 -0.89026066  0.16075562] [-1.42810882 -0.72699012  0.10365747]
[ 0.         0.        81.6770984] [ 0.          0.         38.10937269]
Cp error:  1367.3760096634594 980.8908612367126
Inter-prism error:  842.8160968696009 918.9352663264109
Speed:  0.2
Selected points:  47
4-DOF
Time [s]:  0.2307579517364502
Results done !
[ 3.26168379 -0.53142856 -0.14512302] [-2.72892114 -0.53127186 -0.32976433]
[ 0.         0.        82.2834976] [ 0.          0.         44.03344974]
Cp error:  2449.940184157487 1366.341875975878
Inter-prism error:  1670.854025459463 1748.7679479045878
Speed:  0.3
Selected points:  71
4-DOF
Time [s]:  0.3104584217071533
Results done !
[ 3.32844257 -0.76180214  0.38847138

In [ ]:
print(np.array(Tf_results_bis[index_error_bis]))

In [ ]:
T12_perturbation = np.array([[0.1542515, 0.9880316, 0, 10], [-0.9880316, 0.1542515, 0, -5],
                             [0, 0, 1, -3], [0, 0, 0, 1]])
T13_perturbation = np.array([[0.6333192, 0.7738907, 0, -6], [-0.7738907, 0.6333192, 0, 4],
                             [0, 0, 1, 1], [0, 0, 0, 1]])

print(np.linalg.inv(T12_perturbation))
print(np.linalg.inv(T13_perturbation))